# Assignment: Create a retrieval augmented chain to accept search terms and return results from your vector db or in memory dictionary

In [0]:
!pip install langchain==0.3.0 langchain-chroma==0.1.4 langchain-community==0.3.0 langchain-core==0.3.0 langchain-huggingface==0.1.0 langchain-text-splitters==0.3.0 pypdf

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━

In [0]:
dbutils.library.restartPython() 

In [0]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

## 1. Set Up models & Vector Store

In [0]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN "] = ""

In [0]:
# Create a Chroma vector store
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
)

# Create the LLM
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    huggingfacehub_api_token=os.environ.get("HUGGINGFACEHUB_API_TOKEN "),
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    temperature=0.01,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_llm = ChatHuggingFace(llm=llm, verbose=True)

## 2. Adding document corpus to Vector Store

In [0]:
from langchain_community.document_loaders import PyPDFLoader

def get_docs(path):
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    return pages

In [0]:
docs = []

for i in dbutils.fs.ls("/Volumes/dbx_genai_classroom/rag/retreival_docs/"):
    if "vision_transformers" not in i.path and i.path.endswith("pdf"):
        docs += get_docs(i.path.replace('dbfs:', ''))

In [0]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

In [0]:
vector_store = Chroma(
    collection_name=f"ArunachalPradesh",
    embedding_function=embeddings,
)

**Note**: Run this cell only once to initialize the vector store

In [0]:
vector_store.add_documents(splits)

['874a78b8-79bc-4411-b0df-253382805976',
 'c12b7374-0d8f-4635-aab4-f5a4059f591a',
 '5e3a8205-5380-4a94-9e4a-b676b2b5fca8',
 '57fcd5ea-a459-4152-aa4d-ae6cc1ab65e9',
 '0cb2ce17-267e-4b49-8dc7-f1572651f718',
 '45a733ce-8a05-48ef-b45c-68d743f0db4d',
 '5a0eba32-9fc5-4727-a177-348d3749e6b5',
 '7f52c70f-5f7e-404c-aa9e-0e967dfd1be5',
 'f288bef9-c414-432d-a2bf-6f424cba4b6d',
 '1bc51fb9-786c-403e-8601-88caff7f3f45',
 '2fbe3392-cee1-42c7-a2b9-56a888a692aa',
 '24180e8e-3a79-4483-b775-f1635ad05822',
 'b28fcbbc-c1b0-49f6-855c-8fb558aa64df',
 '5e6c3174-5bee-4bf4-9f61-dd514ffe7a81',
 '40f3bef7-52ba-4491-a9d8-876c24d4833b',
 'd4f78d3e-697b-47bf-bb2a-923da2878c22',
 'cd59d17d-d8e8-4061-8481-beb61c3a79b8',
 '55eaea13-1cce-48bd-907e-45b37e7ba663',
 '479da038-570c-4eea-9f1e-3865f76a2c6a',
 '6e842c0a-7334-466d-b608-d5fbe2a372c6',
 '04bca22e-dce8-4806-b998-d857db22c8da',
 '3fd20bb2-f19d-4b30-977c-4223088f1f9e',
 'a0f9e88e-59d7-41eb-977a-464b9fe77de8',
 '3de6a62c-dd08-48f8-a879-5d011cb446fe',
 '459dc528-4f51-

## 3. Augemented Text Generation

In [0]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [0]:
# Contextualize question #
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate one standalone question which can be understood "
    "without the chat history. Ensure not to loose the semantic & syntactic meaning of the question."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

retriever = vector_store.as_retriever()
print("Initialized Retriever")

history_aware_retriever = create_history_aware_retriever(
    chat_llm, retriever, contextualize_q_prompt
)

print("Created History aware Retriever")

# Answer question #
system_prompt = (
    "You are an expert on the state of Arunchal Pradesh adept at question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain
)

print("Created RAG Chain")

Initialized Retriever
Created History aware Retriever
Created RAG Chain


In [0]:

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [0]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [0]:
generated_content = conversational_rag_chain.invoke(
    {"input": "What are the primary animals found in Arunchal Pradesh?"},
    config={
        "configurable": {"session_id": "uniqueid001"},
    },
)